In [ ]:
##  capture Image from camera and create dataset

In [13]:
import cv2
import os

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    cap = cv2.VideoCapture(0)
    id = 3  # here you can change id for uniqe user
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            data_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(data_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)

        if cv2.waitKey(1) == 13 or img_id == 200:  # 13 is Enter key
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting sample pictures completed.")

generate_dataset()


In [24]:
!pip install pillow

In [ ]:
!pip uninstall opencv-python

In [ ]:
!pip install opencv-contrib-python

In [15]:
import cv2
import os
import numpy as np
from PIL import Image

def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Convert to grayscale
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])  # Extract ID
        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")  # Save the trained model

train_classifier("data")


In [17]:
import cv2

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    coords = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        id, pred = clf.predict(gray_image[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))

        if confidence > 77:
            if id == 1:
                name = "Amarjeet"
            elif id == 2:
                name = "dhruv"
            elif id == 3:
                name = "Krishna"
            else:
                name = "Unknown"
        else:
            name = "Unknown"

        cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords

def recognize(img, clf, faceCascade):
    draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
    return img

# Load classifier and recognizer
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Start video capture
video_capture = cv2.VideoCapture(0)
while True:
    ret, img = video_capture.read()
    if not ret:
        break
    img = recognize(img, clf, faceCascade)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) == 13:  # 13 is Enter key
        break

video_capture.release()
cv2.destroyAllWindows()
